In [1]:
import pandas as pd
import os

In [2]:
folder_path = 'Pneuf_Open_ET_HUC12'

In [3]:
# Define models and rename map
models = ['ensb', 'sebal', 'sims', 'ptjpl', 'alexi', 'sseb', 'metric']
rename_map = {
    'mean': 'mean',
    '25th': '25th',
    '50th': '50th',
    '75th': '75th'
}
years = list(range(1999,2024))
# Dictionary to hold merged per-model DataFrames
model_dfs = {}

# Step 1: Loop over models and merge yearly CSVs
for model in models:
    year_list = years if model != 'alexi' else years[2:]
    # Load and combine all yearly CSVs
    combined_df = pd.concat([pd.read_csv(os.path.join(folder_path,f'OpenET_{model}_monthly_stat_{year}.csv')) for year in year_list], 
                        ignore_index=True)
    
    # Step 2: Rename columns to include model name prefix
    rename_columns = {
        col: f'{model}_{rename_map[col]}'
        for col in rename_map.keys()
        if col in combined_df.columns
    }
    combined_df.rename(columns=rename_columns, inplace=True)
    
    # Keep only necessary columns
    keep_cols = ['Date', 'huc12'] + list(rename_columns.values())
    combined_df = combined_df[keep_cols]
    print(f'{model} columns after rename: {combined_df.columns.tolist()}')
    
    # Save to dictionary
    model_dfs[model] = combined_df

ensb columns after rename: ['Date', 'huc12', 'ensb_mean', 'ensb_25th', 'ensb_50th', 'ensb_75th']
sebal columns after rename: ['Date', 'huc12', 'sebal_mean', 'sebal_25th', 'sebal_50th', 'sebal_75th']
sims columns after rename: ['Date', 'huc12', 'sims_mean', 'sims_25th', 'sims_50th', 'sims_75th']
ptjpl columns after rename: ['Date', 'huc12', 'ptjpl_mean', 'ptjpl_25th', 'ptjpl_50th', 'ptjpl_75th']
alexi columns after rename: ['Date', 'huc12', 'alexi_mean', 'alexi_25th', 'alexi_50th', 'alexi_75th']
sseb columns after rename: ['Date', 'huc12', 'sseb_mean', 'sseb_25th', 'sseb_50th', 'sseb_75th']
metric columns after rename: ['Date', 'huc12', 'metric_mean', 'metric_25th', 'metric_50th', 'metric_75th']


In [12]:
for model, model_df in model_dfs.items():
    model_df.to_csv(f'openet_huc12_{model}.csv',index=False)

In [6]:
for model, model_df in model_dfs.items():
    # Merge with the previous model's DataFrame
    if model == 'ensb':
        merged_df = model_df
    else:
        merged_df = pd.merge(merged_df, model_df, on=['Date', 'huc12'], how='outer')
    
    print(f'Merged {model} into combined DataFrame with shape: {merged_df.shape}')

Merged ensb into combined DataFrame with shape: (13386, 6)
Merged sebal into combined DataFrame with shape: (13386, 10)
Merged sims into combined DataFrame with shape: (13386, 14)
Merged ptjpl into combined DataFrame with shape: (13386, 18)
Merged alexi into combined DataFrame with shape: (13386, 22)
Merged sseb into combined DataFrame with shape: (13386, 26)
Merged metric into combined DataFrame with shape: (13386, 30)


In [7]:
merged_df

,Date,huc12,ensb_mean,ensb_25th,ensb_50th,ensb_75th,sebal_mean,sebal_25th,sebal_50th,sebal_75th,...,alexi_50th,alexi_75th,sseb_mean,sseb_25th,sseb_50th,sseb_75th,metric_mean,metric_25th,metric_50th,metric_75th
0,1999-10-01,170402080608,42.026444,16.000000,30.059822,70.000000,28.041041,6.019110,21.000000,58.00000,...,NaN,NaN,48.646364,11.024615,37.028483,89.00000,49.580417,16.000000,42.000000,78.000000
1,1999-10-01,170402080102,26.512871,18.000000,26.000000,34.000000,18.490520,8.229786,17.000000,28.37175,...,NaN,NaN,30.227945,11.000000,26.415513,46.00000,23.308316,12.300256,21.366088,32.195837
2,1999-10-01,170402080104,30.285855,16.000000,29.000000,42.907922,20.160616,4.000000,16.133722,33.00000,...,NaN,NaN,38.517728,12.099540,34.000000,60.00000,22.781635,9.055931,18.000000,32.000000
3,1999-10-01,170402080106,24.470616,13.476289,22.462536,33.482824,13.929165,4.460521,11.473327,22.49167,...,NaN,NaN,26.359794,6.000000,22.000000,42.00000,20.717864,7.000000,15.000000,27.469061
4,1999-10-01,170402080107,26.152002,17.000000,25.000000,34.000000,12.648812,3.000000,11.000000,19.00000,...,NaN,NaN,27.575865,8.000000,22.000000,40.00000,27.614493,11.000000,23.000000,41.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13381,2023-12-01,170402080401,6.958857,4.000000,6.000000,10.000000,6.613775,0.000000,1.000000,13.00000,...,7.000000,12.479703,6.975248,4.000000,6.000000,10.00000,1.646916,0.000000,1.000000,2.291915
13382,2023-12-01,170402080403,5.467448,3.000000,4.000000,7.000000,4.827527,0.000000,0.000000,9.00000,...,5.000000,7.000000,7.039569,4.000000,6.000000,9.00000,3.649094,1.000000,2.000000,5.000000
13383,2023-12-01,170402080205,10.241848,8.658774,10.000000,11.398277,9.108799,1.000000,8.532289,18.00000,...,12.479907,16.000000,8.117858,4.011419,7.000000,11.00000,2.384249,0.000000,1.000000,3.000000
13384,2023-12-01,170402080402,4.782044,3.000000,4.000000,5.000000,1.460260,0.000000,0.000000,0.00000,...,6.000000,8.000000,6.384312,3.000000,5.000000,8.00000,3.411171,0.000000,3.000000,5.000000


In [8]:
cols = [col for col in merged_df.columns if col in ['Date', 'huc12'] or col.endswith('mean')]  # Display columns of interest

In [9]:
cols

['Date',
 'huc12',
 'ensb_mean',
 'sebal_mean',
 'sims_mean',
 'ptjpl_mean',
 'alexi_mean',
 'sseb_mean',
 'metric_mean']

In [10]:
df_mean = merged_df[cols].copy()

In [11]:
df_mean

,Date,huc12,ensb_mean,sebal_mean,sims_mean,ptjpl_mean,alexi_mean,sseb_mean,metric_mean
0,1999-10-01,170402080608,42.026444,28.041041,28.943985,39.911444,NaN,48.646364,49.580417
1,1999-10-01,170402080102,26.512871,18.490520,32.791097,33.195032,NaN,30.227945,23.308316
2,1999-10-01,170402080104,30.285855,20.160616,48.614720,38.381079,NaN,38.517728,22.781635
3,1999-10-01,170402080106,24.470616,13.929165,36.829052,34.803774,NaN,26.359794,20.717864
4,1999-10-01,170402080107,26.152002,12.648812,29.883294,34.669593,NaN,27.575865,27.614493
...,...,...,...,...,...,...,...,...,...
13381,2023-12-01,170402080401,6.958857,6.613775,13.948754,8.504706,8.671903,6.975248,1.646916
13382,2023-12-01,170402080403,5.467448,4.827527,15.164764,5.874018,5.271535,7.039569,3.649094
13383,2023-12-01,170402080205,10.241848,9.108799,14.915002,16.996438,11.904131,8.117858,2.384249
13384,2023-12-01,170402080402,4.782044,1.460260,11.349560,5.358675,6.204585,6.384312,3.411171


In [13]:
df_mean.to_csv('openet_huc12_mean.csv', index=False)